# DTI ROI Analysis:
### Requirements:
* Excel sheet with DTI metrics from ROI script by PIDN and JHU atlas label
* Excel sheet with covariates (age, sex, biomarkers) by PIDN
* Excel sheet with JHU Atlas labels and abbreviations (to make reading the graphs easier)

### Initial code:
* Kevin Chang, 8/2018

In [2]:
import pandas as pd
import os
import numpy as np
import xlsxwriter
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pprint import pprint
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import BoxAnnotation, NumeralTickFormatter, Legend
from bokeh.palettes import Blues5
import math
output_notebook()

Loading BokehJS ...

In [3]:
## Function builds the input for statsmodels
# default settings: No scanner selection, FA for metric, no PIDNs dropped
# Must provide:
#    input_df w/ dti metrics
#    covariate_df with other variables
#    ROI name to look at
#    covariates to input into GLM
#    Options:
#        Scanner: to restrict by scanner, ie Prisma vs Trio
#        Metric: defaults to "FA"
#        drop: to discard any PIDNs, ie outliers

def create_dataframes(input_df, covariate_df,ROI,covariates,Scanner=None,Metric="FA",drop=None):
    if Scanner:
        target_df = input_df.loc[
            (input_df["Scanner"]==Scanner) &
            (input_df["Metric"]==Metric)
        ][ROI]
        
        covariate = covariate_df.loc[
            (input_covariates["Scanner"]==Scanner)
        ][
            covariates
        ]
    else:
        target_df = input_df.loc[
            (input_df["Metric"]==Metric)
        ][ROI]
        covariate = covariate_df[
            covariates
        ]
    X = covariate
    Y = target_df
    data_ = pd.concat([X,Y],axis=1)
    if drop:
        data_ = data_.drop(drop)
    #Y=Y.index.name=None
    return X,Y,data_

## Step 1: Reading through the input excel sheets and setting variables:

In [24]:
## path = Directory to work inside
path="R://groups//valcour//kevin//02_Local//sCD163_and_neop//analysis//diffusion//"
os.chdir(path)

## Building the JHU atlas dictionary from file w/ abbreviations to make working easier
atlas_names_path = "R://groups//valcour//kevin/Atlas_labels.xlsx"
atlas_names = pd.read_excel(atlas_names_path, sheet_name = "JHU", index_col = "Name")
atlas_dict={}
for i in atlas_names.index:
    atlas_dict[i] = atlas_names.loc[i,"Abbr"]

## unified_xlx = excel file with the DTI ROI output
unified_xlx = "unified_analysis_wkbk_.xlsx"
## unified_sheet = sheet name to look at
unified_sheet = "combined"
# Read the ROI metric file to dataframe:
input_df = pd.read_excel(unified_xlx, sheet_name = unified_sheet, index_col="PIDN")
# Rename the atlas labels to abbreviations here:
input_df = input_df.rename(atlas_dict,axis=1)


## cov_path = excel sheet with the desired covariates by PIDN
cov_path = "R://groups//valcour//kevin//02_Local//sCD163_and_neop//analysis//covariates.xlsx"
# Read the covariate file, dropping any PIDNs as necessary
input_covariates = pd.read_excel(cov_path,index_col="PIDN")
PIDNs2discard = [10317]
input_covariates=input_covariates.drop(PIDNs2discard)

## Create list of ROIs to examine
ROIs = input_df.columns[3:]
ROIs2 = ["GCC","BCC","SCC","ACR_R","ACR_L","SCR_R","SCR_L","PCR_R","PCR_L","SLF_R","SLF_L"]

## Create list of covariates to examine
covariates = [
    "Age_screen","Educ_yrs", "Days_between","Sex","Sex_","Race_white","Race","Diag","Scanner","Yr_infected",
    "CD4_count", "CD4_nadir", "CD8_count", "CD4_CD8_ratio",
    "MCP1","CD163","IP10","Neopterin","CD14"
]
       
print "DONE"

DONE


In [173]:
## To run GLM on only one ROI for testing
roi = "GCC"
var2test = "MCP1"
other_covs = cov__

X,Y,data_ = create_dataframes(input_df,input_covariates,Scanner="Prisma",Metric="MD",ROI=roi,covariates=covariates,drop=None)

# Convert categorical data to codes here:
data_["Scanner__"] = pd.Categorical(data_["Scanner"]).codes
data_["Race__"] = pd.Categorical(data_["Race"]).codes

if False:
    # For debugging
    print data_.head()

est = smf.glm(
    formula='%s ~ %s + %s' % (
        roi,
        var2test,
        other_covs,
    ),
    data = data_
).fit()

print "Model built with ROI:",roi
print "Variable of interest:",var2test
print "Other covariates included in model:",other_covs.split("+"),"\n"
print est.summary()


Model built with ROI: GCC
Variable of interest: MCP1
Other covariates included in model: ['Age_screen', 'Sex'] 

                   Generalized Linear Model Regression Results                   
Dep. Variable:                    GCC   No. Observations:                      20
Model:                            GLM   Df Residuals:                          16
Model Family:                Gaussian   Df Model:                               3
Link Function:               identity   Scale:              0.0012882312362990228
Method:                          IRLS   Log-Likelihood:                    40.398
Date:                Wed, 29 Aug 2018   Deviance:                        0.020612
Time:                        16:42:32   Pearson chi2:                      0.0206
No. Iterations:                     2                                            
                  coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------

In [42]:
## Loop to test multiple variables against the same covariates at multiple ROIs

#vars2test = ["MCP1","CD163","IP10","Neopterin","CD14"]
vars2test = ["MCP1"]
#vars2test = ["CD4_CD8_ratio"]

ROIs_ = ROIs2

## Setting GLM options:
scanner = "Trio"
#scanner = "Prisma"

#metrics_ = ["FA","MD","RD"]
metrics_ = ["MD"]


## Threshold for graphing:
threshold = 0.05
threshold2 = 0.01
threshold3 = 0.001

ROI_dict = {}

## Building covariate input to use in model
covariate_options = [
    "Age_screen",
    #"Educ_yrs",
    #"Sex",
    #"Race",
    #"CD4_count",
    #"CD8_count",
    #"CD4_CD8_ratio",
    #"Diag",
]
cov__ = ""
for i in covariate_options:
    cov__ += i
    if i != covariate_options[-1]:
        cov__ += "+"

other_covs = cov__
#other_covs = "Age_screen"
#other_covs = None


print "Testing",len(ROIs_),"ROI(s):", ROIs_
print "Testing individually across",len(vars2test),"variable(s) of interest:",vars2test
if other_covs:
    print "Testing against",len(other_covs.split("+")),"covariate(s):", other_covs.split("+"), "\n"
else:
    print "No other covariates included in model\n"

## For each variable of interest in vars2test
for var in vars2test:
    x = []
    y = []
    ## In each roi in ROIs_:
    for roi in ROIs_:
        ## And for each metric in metrics_
        for metric in metrics_:
            ## Create the dataframe to run GLM on
            X,Y,data_ = create_dataframes(
                input_df,input_covariates,Scanner=scanner,Metric=metric,ROI=roi,covariates=covariates
            )
            ## Categorical data converted to numerical codes here:
            #data_["Race__"] = pd.Categorical(data_["Race"]).codes
            #data_["Diag__"] = pd.Categorical(data_["Diag"]).codes

            ## For if we want to look at specific diagnoses, etc...
            #data_ = data_.loc[data_["Diag"]=="MND"]
            data_ = data_.loc[data_["Diag"]!="NL"]
            #data_ = data_.loc[data_["Sex"]=="Male"]
            #data_ = data_.loc[data_["Race"]=="White"]
            
            ## 1. For Biomarker + all covs in other_covs
            if other_covs:
                est = smf.glm(
                    formula='%s ~ %s + %s' % (roi, var, other_covs),
                    data = data_
                ).fit()

            ## 2. For single variable set as var
            else:
                est = smf.glm(
                    formula='%s ~ %s' % (roi, var),    
                    data = data_
                ).fit()
            if True:
                print roi, metric
                print est.summary()
                print ""
                print ""

            

            ROI_dict["%s, %s" % (roi, metric)] = est.pvalues.to_dict()
            
            x.append("%s, %s" % (roi, metric))
            y.append(ROI_dict["%s, %s" % (roi, metric)][var])


    #for i in ROI_dict.keys():
    #    x.append(i)
    #    y.append(ROI_dict[i][var])

    xs = sorted(x)

    x=np.asarray(x)
    y=np.asarray(y)

    print var,":",len(y[y<=threshold]),"/",len(y), "ROIs are significant at p <",threshold
    print np.sort(x[y<=threshold])
    
    ## matplotlib style output to see significance across ROI and metric
    if False:
        col = np.where(y<=threshold,"blue","red")
        plt.figure(figsize=(len(y),5))
        #plt.figure(figsize=(30,5))
        if True:
            plt.scatter(x,y,c=col,marker="x")
            plt.plot([xs[0],xs[-1]],[threshold,threshold],"k--",alpha=0.5)
        else:
            plt.plot(x[y<=threshold],y[np.abs(y<=threshold)] , "bx")
            plt.plot(x[y>threshold],y[np.abs(y>threshold)] , "rx")
        plt.xlabel("Metric and ROI")
        plt.xticks(rotation=90)
        plt.title("GLM results for %s" % cov)
        plt.ylabel("p-value from GLM")

        ## Save image
        if False:
            plt.savefig("%s_GLM.png" % cov)
        plt.show()
    
    # bokeh style output to see significance across ROI and metric
    if True:
        #p = figure(plot_width=900, plot_height=400, x_range = xs, y_range = [0,y.max()])
        #x_wid = len(xs)*75
        title = "%s: %s/%s ROIs on %s are significant at p<%s" % (var,len(y[y<=threshold]),len(y),scanner,threshold)
        
        p = figure(plot_width=900, plot_height=400,
                   x_range = x, y_axis_type = "log",y_range = (0.00001,1),toolbar_location="right",tools="save", title=title
                  )
        p.background_fill_color = "beige"
        p.background_fill_alpha = 0.5
        p.yaxis.ticker = (0,0.001 ,0.01, 0.05, 0.1, 0.25, 0.5,1)
        p.yaxis[0].formatter = NumeralTickFormatter(format="0.0[000]")
        p.yaxis.major_tick_out = 10
        p.yaxis.axis_label = "p-value from GLM"
        #p.ygrid.grid_line_color = None
        #p.ygrid.bounds = (0,0.1)
        p.xaxis.axis_label = "ROI,metric"
        p.xaxis.major_label_orientation = math.pi/2
        
        
        
        
        sig_lines_x = [xs[0]]*6
        sig_lines_y = sorted([threshold,threshold2,threshold3]*2)
        angles = [0,180]*3
        colors = ["mediumblue"]*2+["cornflowerblue"]*2+["lightsteelblue"]*2
        p.ray(
            x=sig_lines_x,y=sig_lines_y,length=100, angle=angles,angle_units = "deg",
            line_color=colors,line_width=2
        )
        
        #p.rect(x=[xs[0]],y=[0.525],height=0.95,width=100,fill_color="gainsboro",alpha=0.75)
        
        p1= p.circle(x[y>threshold],y[np.abs(y>threshold)],size=10, fill_color="peachpuff",line_color="gray",line_width=1,alpha=1)
        p2= p.circle(x[y<=threshold],y[np.abs(y<=threshold)],size=10, fill_color=colors[-1],line_color="black",line_width=1)
        p3= p.circle(x[y<=threshold2],y[np.abs(y<=threshold2)],size=10, fill_color=colors[3],line_color="black",line_width=1)
        p4= p.circle(x[y<=threshold3],y[np.abs(y<=threshold3)],size=10, fill_color=colors[0],line_color="black",line_width=1)
        
        legend = Legend(items=[
            ("p>%s"%threshold, [p1]),
            ("p<%s"%threshold, [p2]),
            ("p<%s"%threshold2, [p3]),
            ("p<%s"%threshold3, [p4])
        ], location=(0,150),background_fill_color = "white",background_fill_alpha=0.5)
        p.add_layout(legend,"right")
        show(p)
    






Testing 11 ROI(s): ['GCC', 'BCC', 'SCC', 'ACR_R', 'ACR_L', 'SCR_R', 'SCR_L', 'PCR_R', 'PCR_L', 'SLF_R', 'SLF_L']
Testing individually across 1 variable(s) of interest: ['MCP1']
Testing against 1 covariate(s): ['Age_screen'] 

GCC MD
                  Generalized Linear Model Regression Results                   
Dep. Variable:                    GCC   No. Observations:                     19
Model:                            GLM   Df Residuals:                         16
Model Family:                Gaussian   Df Model:                              2
Link Function:               identity   Scale:              0.002286053812257076
Method:                          IRLS   Log-Likelihood:                   32.442
Date:                Thu, 30 Aug 2018   Deviance:                       0.036577
Time:                        11:35:41   Pearson chi2:                     0.0366
No. Iterations:                     2                                           
                 coef    std err      